# Data collected for Group 20:

1. Timeseries Climate Data 
- temp
- rainfall

2. Shape Data 
- Australian Shape file (states)

3. Crop Reports (area, production & forcasts)


In [1]:
import pandas as pd
import os
import gzip
import os
import unlzw3
import pandas as pd
from pathlib import Path

import matplotlib as plt
import numpy as np 
import boto3
import pandas as pd
import os

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import datetime

# Temp Time series data

acorn_sat_v2.5.0_daily_tmax
acorn_sat_v2.5.0_daily_tmean
acorn_sat_v2.5.0_daily_tmin

In [2]:
basedata = pd.read_csv('/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CLIMATE/TEMP/acorn_sat_v2.5.0_daily_tmax/tmax.001019.daily.csv')

In [3]:
folders = {
    'tmax': '/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CLIMATE/TEMP/acorn_sat_v2.5.0_daily_tmax',
    'tmean': '/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CLIMATE/TEMP/acorn_sat_v2.5.0_daily_tmin',
    'tmin': '/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CLIMATE/TEMP/acorn_sat_v2.5.0_daily_tmean'
}

data_frames = {'tmax': [], 'tmean': [], 'tmin': []}

In [4]:
def load_and_clean_csv(file_path):

    df = pd.read_csv(file_path)
    
    
    df['site number'] = df['site number'].ffill()  # forward fill 'site number'
    df['site name'] = df['site name'].ffill()      
    
    return df

In [5]:
for key, folder in folders.items():
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            file_path = os.path.join(folder, file)
            df = load_and_clean_csv(file_path)
            data_frames[key].append(df)


df_tmax = pd.concat(data_frames['tmax'], ignore_index=True)
df_tmean = pd.concat(data_frames['tmean'], ignore_index=True)
df_tmin = pd.concat(data_frames['tmin'], ignore_index=True)

# Merge the dataframes on 'date' and 'site number'
temp_df = pd.merge(df_tmax, df_tmean, on=['date', 'site number', 'site name'], suffixes=('_tmax', '_tmean'))
temp_df = pd.merge(temp_df, df_tmin, on=['date', 'site number', 'site name'], suffixes=('', '_tmin'))


temp_df.head()

,date,maximum temperature (degC),site number,site name,minimum temperature (degC),mean temperature (degC)
0,NaN,NaN,9999.0,ALBANY AIRPORT,NaN,NaN
1,1910-01-01,23.9,9999.0,ALBANY AIRPORT,15.2,19.5
2,1910-01-02,20.3,9999.0,ALBANY AIRPORT,14.8,17.5
3,1910-01-03,25.0,9999.0,ALBANY AIRPORT,15.2,20.1
4,1910-01-04,21.5,9999.0,ALBANY AIRPORT,12.9,17.2


In [6]:
temp_df = temp_df.dropna(subset=['maximum temperature (degC)', 'minimum temperature (degC)', 'mean temperature (degC)'], how='all')

In [7]:
len(temp_df['site number'].unique())

112

In [8]:
temp_df['date'] = pd.to_datetime(temp_df['date'], format='%Y-%m-%d')

In [9]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3997047 entries, 1 to 4063701
Data columns (total 6 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   date                        datetime64[ns]
 1   maximum temperature (degC)  float64       
 2   site number                 float64       
 3   site name                   object        
 4   minimum temperature (degC)  float64       
 5   mean temperature (degC)     float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 213.5+ MB


In [10]:
temp_df.head()

,date,maximum temperature (degC),site number,site name,minimum temperature (degC),mean temperature (degC)
1,1910-01-01,23.9,9999.0,ALBANY AIRPORT,15.2,19.5
2,1910-01-02,20.3,9999.0,ALBANY AIRPORT,14.8,17.5
3,1910-01-03,25.0,9999.0,ALBANY AIRPORT,15.2,20.1
4,1910-01-04,21.5,9999.0,ALBANY AIRPORT,12.9,17.2
5,1910-01-05,22.6,9999.0,ALBANY AIRPORT,10.8,16.7


In [53]:
temp_df.to_csv('Temp_Data.csv', index=False)

## RainFall Data

Data derived from http://www.bom.gov.au/climate/change/hqsites/about-hq-site-data.shtml

Australian daily rainfall dataset.

timeseries data from
151 stations

## Station Metadata

* Station Number
* Lat
* Long
* Elevation
* Station Name

In [11]:
def read_stations_file(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    cleaned_data = []
    for line in lines:
        
        station_number = line[0:6].strip()    
        latitude = line[7:13].strip()        
        longitude = line[14:20].strip()       
        elevation = line[21:27].strip()       
        station_name = line[28:].strip()      

        cleaned_data.append([station_number, latitude, longitude, elevation, station_name])

    
    stations_df = pd.DataFrame(cleaned_data, columns=['Station Number', 'Latitude', 'Longitude', 'Elevation', 'Station Name'])
    
    return stations_df

In [12]:
rain_stations_file = '/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CLIMATE/DAILYRAIN/HQDR_stations.txt'
rain_stations_df = read_stations_file(rain_stations_file)

In [13]:
rain_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Station Number  151 non-null    object
 1   Latitude        151 non-null    object
 2   Longitude       151 non-null    object
 3   Elevation       151 non-null    object
 4   Station Name    151 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [52]:
rain_stations_df.to_csv('rain_stations_df.csv', index=False)


## Data

* Station Number
* Date
* Rainfall 

Daily rainfall totals are for the 24-hours to 9am (local time). However, the dataset contains some multi-day totals of rainfall. These are identifiable as totals which follow one or more days of missing data (99999.9). Any rainfall recorded on the first day following a period of missing data may have fallen on any of the previous days with missing data or may have been recorded over more than one day.
Throughout the data files 99999.9 is used to denote missing data.

In [14]:
rainfall_folder = '/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CLIMATE/DAILYRAIN/HQ_daily_prcp_txt/'

In [15]:
def read_rainfall_data(file_path, station_number):
    
    with file_path.open('rb') as f:
        uncompressed_data = unlzw3.unlzw(f.read())
    
    
    uncompressed_str = uncompressed_data.decode('utf-8')
    
    # Split and skip the header (first line)
    lines = uncompressed_str.split('\n')[1:]
    
    
    rainfall_data = []
    for line in lines:
        if line.strip():  
            date_str, rainfall = line.split()
            rainfall_data.append([station_number, date_str, float(rainfall)])
    
    
    rainfall_df = pd.DataFrame(rainfall_data, columns=['Station Number', 'Date', 'Rainfall (mm)'])
    
    return rainfall_df

In [16]:
station_files = [f for f in Path(rainfall_folder).glob('*.Z')] # .Z files 

all_station_data = {}

for station_file in station_files:
    
    station_number = station_file.stem.split('.')[1]  # Extract station number from the file name, e.g., "007057"
    rainfall_df = read_rainfall_data(station_file, station_number)
    all_station_data[station_number] = rainfall_df

combined_df = pd.concat(all_station_data.values(), ignore_index=True)

In [17]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7142475 entries, 0 to 7142474
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Station Number  object 
 1   Date            object 
 2   Rainfall (mm)   float64
dtypes: float64(1), object(2)
memory usage: 163.5+ MB


In [18]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%Y%m%d')

In [19]:
combined_df.tail()

,Station Number,Date,Rainfall (mm)
7142470,010525,2021-11-19,0.0
7142471,010525,2021-11-20,0.0
7142472,010525,2021-11-21,0.0
7142473,010525,2021-11-22,0.0
7142474,010525,2021-11-23,16.5


In [20]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7142475 entries, 0 to 7142474
Data columns (total 3 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Station Number  object        
 1   Date            datetime64[ns]
 2   Rainfall (mm)   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 163.5+ MB


In [21]:
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Month'] = combined_df['Date'].dt.month

In [22]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7142475 entries, 0 to 7142474
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Station Number  object        
 1   Date            datetime64[ns]
 2   Rainfall (mm)   float64       
 3   Year            int32         
 4   Month           int32         
dtypes: datetime64[ns](1), float64(1), int32(2), object(1)
memory usage: 218.0+ MB


# MERGED DATA!
rainfall with meta data

In [23]:
merged_df = pd.merge(combined_df, rain_stations_df, on='Station Number')

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7142475 entries, 0 to 7142474
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Station Number  object        
 1   Date            datetime64[ns]
 2   Rainfall (mm)   float64       
 3   Year            int32         
 4   Month           int32         
 5   Latitude        object        
 6   Longitude       object        
 7   Elevation       object        
 8   Station Name    object        
dtypes: datetime64[ns](1), float64(1), int32(2), object(5)
memory usage: 435.9+ MB


In [54]:
merged_df.head()

,Station Number,Date,Rainfall (mm),Year,Month,Latitude,Longitude,Elevation,Station Name
0,009503,1903-02-01,0.0,1903,2,-33.48,115.73,0037.0,BOYANUP POST OFFICE
1,009503,1903-02-02,0.0,1903,2,-33.48,115.73,0037.0,BOYANUP POST OFFICE
2,009503,1903-02-03,0.0,1903,2,-33.48,115.73,0037.0,BOYANUP POST OFFICE
3,009503,1903-02-04,0.0,1903,2,-33.48,115.73,0037.0,BOYANUP POST OFFICE
4,009503,1903-02-05,0.0,1903,2,-33.48,115.73,0037.0,BOYANUP POST OFFICE


In [56]:
merged_df.to_csv('rainfall.csv', index=False)

## State shape files

https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

In [25]:
file = "/Users/brydejeffery/Desktop/GROUPPROJECT/Data/SHAPEFILE/SOS_2021_AUST_GDA94_SHP/SOS_2021_AUST_GDA94.shp"
states_gdf = gpd.read_file(file)

In [26]:
states_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   SOS_CODE21  53 non-null     object  
 1   SOS_NAME21  53 non-null     object  
 2   STE_CODE21  53 non-null     object  
 3   STE_NAME21  53 non-null     object  
 4   AUS_CODE21  53 non-null     object  
 5   AUS_NAME21  53 non-null     object  
 6   AREASQKM21  34 non-null     float64 
 7   LOCI_URI21  53 non-null     object  
 8   geometry    34 non-null     geometry
dtypes: float64(1), geometry(1), object(7)
memory usage: 3.9+ KB


In [27]:
states_gdf["STE_NAME21"].unique()

array(['New South Wales', 'Victoria', 'Queensland', 'South Australia',
       'Western Australia', 'Tasmania', 'Northern Territory',
       'Australian Capital Territory', 'Other Territories',
       'Outside Australia'], dtype=object)

In [28]:
geometry = [Point(xy) for xy in zip(rain_stations_df['Longitude'], rain_stations_df['Latitude'])]
stations_gdf = gpd.GeoDataFrame(rain_stations_df, geometry=geometry)

# NEED -> CRS (Coordinate Reference System) of both GeoDataFrames match
stations_gdf = stations_gdf.set_crs(states_gdf.crs)

stations_with_states = gpd.sjoin(stations_gdf, states_gdf[['STE_NAME21', 'geometry']], how='left', predicate='within')

stations_with_states = stations_with_states.rename(columns={'STE_NAME21': 'State'})

stations_with_states = stations_with_states.drop(columns=['geometry'])

In [29]:
print(stations_with_states.head())

  Station Number Latitude Longitude Elevation  Station Name  index_right  \
0         004035   -20.78    117.15    0012.0     ROEBOURNE         26.0   
1         005008   -21.19    115.98    0011.0        MARDIE         29.0   
2         006055   -27.75    115.83    0300.0    WOOLGORONG         29.0   
3         007007   -26.98    116.54    0300.0      BOOLARDY         29.0   
4         007057   -28.06    117.84    0426.0  MOUNT MAGNET         29.0   

               State  
0  Western Australia  
1  Western Australia  
2  Western Australia  
3  Western Australia  
4  Western Australia  


In [30]:
stations_with_states["State"].unique()

array(['Western Australia', 'Northern Territory', 'South Australia',
       'Queensland', nan, 'New South Wales', 'Victoria', 'Tasmania'],
      dtype=object)

# Crop Report Data

<!-- Table 1 Winter crop production, Australia, 1989–90 to 2024–25	

Table 2 Winter crop area, Australia, 1989–90 to 2024–25		

Table 3 Summer crop area and production, Australia, 1989–90 to 2024–25							

Table 4 Winter crop forecasts, New South Wales, 2024–25							

Table 5 Summer crop forecasts, New South Wales, 2024–25							

Table 6 Winter crop forecasts, Queensland, 2024–25							

Table 7 Summer crop forecasts, Queensland, 2024–25							

Table 8 Winter crop forecasts, Victoria, 2024–25							

Table 9 Winter crop forecasts, South Australia, 2024–25							

Table 10 Winter crop forecasts, Western Australia, 2024–25							

Table 11 Winter crop production and area, Australia, 1989–90 to 2024–25							

Table 12 Summer crop production and area, Australia, 1989–90 to 2024–25							

Table 13 Production, major crops, Australian states, 2017–18 to 2024–25							

Table 14 Production, other crops, Australian states, 2017–18 to 2024–25							

Table 15 Supply and disposal of wheat, canola and pulses, Australia, 1989–90 to 2022–23	

Table 16 Supply and disposal of coarse grains, Australia, 1989–90 to 2022–23							

Table 17 Grain, oilseed and pulse prices, first quarter 1995 to second quarter 2024							 -->



In [31]:
cropdatafile = '/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CropReports/AustCropRrt20240903_CropData_v1.0.0.xlsx'
excel_data = pd.ExcelFile(cropdatafile)

sheet_names = excel_data.sheet_names[1:]

In [32]:
print(sheet_names)

['Table 1', 'Table 2', 'Table 3', 'Table 4', 'Table 5', 'Table 6', 'Table 7', 'Table 8', 'Table 9', 'Table 10', 'Table 11', 'Table 12', 'Table 13', 'Table 14', 'Table 15', 'Table 16', 'Table 17']


INCONSISTENT TABLES

Load the entire sheet into a DataFrame then:

1. Remove empty rows from the DataFrame.
2. Extract the title (the first row with non-empty values).
3. Set the next row as the column headers.
4. The remaining rows will be the data.

In [33]:
table_titles ={}
for i, sheet in enumerate(sheet_names, start=1):

    df = pd.read_excel(cropdatafile, sheet_name=sheet, engine='openpyxl')
    df_clean = df.dropna(how='all')
    
    # get title of table
    table_title = df_clean.iloc[0, 1]
    table_titles[f'table{i}_df'] = table_title  # Save the title for mapping (might be useful)-> more informative
    
    df_clean.columns = df_clean.iloc[1]
    
    df_clean = df_clean.drop(df_clean.index[[0, 1]]).reset_index(drop=True)
    
    df_clean = df_clean.dropna(axis=1, how='all')

    globals()[f'table{i}_df'] = df_clean

In [34]:
print("Mapping of tables to titles:")
for table_name, title in table_titles.items():
    print(f"{table_name}: {title}")

Mapping of tables to titles:
table1_df: Table 1 Winter crop production, Australia, 1989–90 to 2024–25
table2_df: Table 2 Winter crop area, Australia, 1989–90 to 2024–25
table3_df: Table 3 Summer crop area and production, Australia, 1989–90 to 2024–25
table4_df: Table 4 Winter crop forecasts, New South Wales, 2024–25
table5_df: Table 5 Summer crop forecasts, New South Wales, 2024–25
table6_df: Table 6 Winter crop forecasts, Queensland, 2024–25
table7_df: Table 7 Summer crop forecasts, Queensland, 2024–25
table8_df: Table 8 Winter crop forecasts, Victoria, 2024–25
table9_df: Table 9 Winter crop forecasts, South Australia, 2024–25
table10_df: Table 10 Winter crop forecasts, Western Australia, 2024–25
table11_df: Table 11 Winter crop production and area, Australia, 1989–90 to 2024–25
table12_df: Table 12 Summer crop production and area, Australia, 1989–90 to 2024–25
table13_df: Table 13 Production, major crops, Australian states, 2018–19 to 2024–25
table14_df: Table 14 Production, other cr

In [35]:
print(table_titles['table1_df'])
table1_df

Table 1 Winter crop production, Australia, 1989–90 to 2024–25


6,Year,Unit,New South Wales,Victoria,Queensland,South Australia,Western Australia,Australia
0,1989–90,kt,4841.223,3343.008,1829.027,4832.81,6606.416,21491.804
1,1990–91,kt,5826.706,2712.227,2486.538,3901.414,7346.772,22325.977
2,1991–92,kt,3909.599,2814.517,461.502,4534.015,7183.333,18962.288
3,1992–93,kt,5950.707,4120.684,1053.865,4797.718,8499.538,24487.185
4,1993–94,kt,7619.149,4479.907,834.394,4923.156,9872.962,27796.784
5,1994–95,kt,1528.143,1863.646,311.546,3044.445,7924.331,14719.274
6,1995–96,kt,6961.863,4500.51,745.907,5263.461,10244.394,27785.136
7,1996–97,kt,11286.144,4590.51,2468.14,5456.971,11192.607,35060.019
8,1997–98,kt,8557.372,3397.664,1639.374,5457.808,11994.754,31112.569
9,1998–99,kt,9717.859,3507.415,2323.341,6303.985,12232.647,34159.264


In [36]:
print(table_titles['table2_df'])
table2_df

Table 2 Winter crop area, Australia, 1989–90 to 2024–25


6,Year,Unit,New South \nWales,Victoria,Queensland,South Australia,Western Australia,Australia
0,1989–90,'000 ha,3076.358,1814.547,1147.332,2831.221,4955.821,13843.934
1,1990–91,'000 ha,3234.096,1866.916,1353.088,2713.525,5160.261,14349.881
2,1991–92,'000 ha,2834.755,1794.777,704.773,2681.68,5018.195,13058.076
3,1992–93,'000 ha,3005.767,1996.138,912.714,2846.786,5497.901,14284.325
4,1993–94,'000 ha,3289.14,2099.858,815.896,2738.004,5945.715,14915.05
5,1994–95,'000 ha,2579.453,2076.104,530.838,2741.614,6033.949,13988.148
6,1995–96,'000 ha,3837.823,2197.453,844.112,2974.72,6244.624,16126.669
7,1996–97,'000 ha,4676.443,2326.249,1225.306,3047.039,6793.986,18097.638
8,1997–98,'000 ha,4543.845,2304.978,1187.732,3024.841,7159.081,18247.356
9,1998–99,'000 ha,4925.747,2453.385,1377.743,3376.304,7420.099,19580.726


In [37]:
print(table_titles['table3_df'])
table3_df

Table 3 Summer crop area and production, Australia, 1989–90 to 2024–25


7,Year,New South Wales,NaN,Queensland,NaN,Australia,NaN
0,NaN,'000 ha,kt,'000 ha,kt,'000 ha,kt
1,1989–90,480.98,1716.239,440.352,941.1,932.181,2675.919
2,1990–91,463.948,1611.872,577.605,994.688,1053.329,2625.233
3,1991–92,542.956,2078.962,658.492,1529.577,1211.172,3627.471
4,1992–93,487.659,1612.114,514.567,640.548,1013.647,2279.798
5,1993–94,516.866,1805.586,651.079,1214.428,1183.259,3052.341
6,1994–95,523.98,1869.23,761.261,1279.379,1305.983,3186.523
7,1995–96,573.725,2076.983,845.565,1535.019,1435.387,3637.955
8,1996–97,654.631,2652.994,746.805,1553.167,1416.335,4239.034
9,1997–98,635.349,2594.515,691.628,1185.607,1340.961,3815.87


In [38]:
print(table_titles['table11_df'])
table11_df

Table 11 Winter crop production and area, Australia, 1989–90 to 2024–25


6,Year,Wheat,NaN,Barley,NaN,Canola,NaN,Chickpeas,NaN,Faba beans,...,Field peas,NaN,Lentils,NaN,Lupins,NaN,Oats,NaN,Triticale,NaN
0,NaN,Area,Production,Area,Production,Area,Production,Area,Production,Area,...,Area,Production,Area,Production,Area,Production,Area,Production,Area,Production
1,NaN,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,...,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,kt
2,1989–90,9003.8,14214.6,2309.897,4044.146,49.736,78.157,93.079,109.368,44.555,...,326.332,388.023,0.33,0.275,801.457,772.018,1088.7,1640.3,91.341,164.372
3,1990–91,9218.2,15066.1,2555.651,4107.832,72.885,98.4,178.121,191.668,43.9,...,318.035,318.068,0.154,0.183,792.561,757.67,1043.6,1529.8,103.96,188.577
4,1991–92,7183,10557.3,2744.241,4529.678,150.665,170.448,249.672,221.859,58.108,...,426.534,472.571,0.293,0.474,938.546,1047.363,1160.1,1689.8,101.02,177.342
5,1992–93,8274.7,14738.7,2947.179,5396.758,107.109,177.902,151.802,176.938,82,...,382.338,456.311,0.479,0.52,1032.473,1194.983,1149.1,1936.4,122,282.477
6,1993–94,8383.1,16479.2,3424.468,6668.234,176.532,305.02,146.837,193.108,93.736,...,399.832,558.453,0.98,1.244,1150.446,1479.652,947.336,1646.499,129.235,262.815
7,1994–95,7890.6,8961.4,2470.49,2913.189,356.234,263.892,208.854,68.893,100.609,...,460.58,240.407,5.203,2.02,1407.066,1076.215,896.569,924.075,153.424,181.581
8,1995–96,9220.8,16503.9,3110.588,5822.684,376.558,557.054,216.415,286.909,80.369,...,381.617,529.919,6.503,15.5,1322.821,1558.995,1136.234,1874.808,221.097,468.513
9,1996–97,10935.5,22924.2,3365.806,6694.804,407.456,623.239,241.45,287.721,96.937,...,336.678,454.615,13.763,28.347,1259.181,1521.861,1051.683,1652.797,346.283,673.774


In [39]:
print(table_titles['table5_df'])
table5_df

Table 5 Summer crop forecasts, New South Wales, 2024–25


6,Crop,Area,Yield,Production,Area change,Prod. change
0,NaN,'000 ha,t/ha,kt,%,%
1,Grain sorghum,180,4.166667,750,2.857143,5.633803
2,Cotton lint,318.5,2.224804,708.6,–5.6,–6.9
3,Cottonseed,318.5,2.550525,812.342,–5.6,–6.9
4,Rice,49,10.5,514.5,–14,–16.7
5,"Notes: Yields are based on area planted, excep...",NaN,NaN,NaN,NaN,NaN


# Global data on crop nutrient concentration and harvest indices

In [40]:
crop_nutr = pd.read_csv("/Users/brydejeffery/Desktop/GROUPPROJECT/Data/CropReports/doi_10_5061_dryad_n2z34tn0x__v20231127/Combined_crop_data.csv")

In [41]:
crop_nutr.head()

,original_crop,item,item_code,item_group_code,item_group,factor,hs_code,hs07_code,hs12_code,cpc_code,...,un_region_name,un_sub_region_name,un_intermediate_region_name,latitude,longitude,reference_where_data_were_collated,website_of_source_of_collated_data,primary_reference_of_dataset,variable,value
0,Agave fibres nes,Agave fibres nes,800.0,1714.0,Crops Primary,1.0,NaN,530500,530500,1929.06,...,NaN,NaN,NaN,NaN,NaN,International Fertilizer Association (2020),https://api.ifastat.org/reports/download/12965,NaN,P_pc_fresh,0.1100
1,Agave fibres nes,Agave fibres nes,800.0,1714.0,Crops Primary,1.0,NaN,530500,530500,1929.06,...,NaN,NaN,NaN,NaN,NaN,International Fertilizer Association (2020),https://api.ifastat.org/reports/download/12965,NaN,K_pc_fresh,0.3320
2,Agave fibres nes,Agave fibres nes,800.0,1714.0,Crops Primary,1.0,NaN,530500,530500,1929.06,...,NaN,NaN,NaN,NaN,NaN,International Fertilizer Association (2020),https://api.ifastat.org/reports/download/12965,NaN,N_pc_fresh,0.5000
3,Alfalfa,NaN,643.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Americas,Northern America,NaN,NaN,NaN,USDA (2009) Chapter 6 Role of Plants in Waster...,https://directives.sc.egov.usda.gov/OpenNonWeb...,Kilmer 1982; Morrison\r\n1956; Sanchez 1976; U...,N_pc,2.2500
4,Alfalfa,NaN,643.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Roy (2016) FAO Fertilizer and Plant Nutrition ...,http://www.fao.org/3/a-a0443e.pdf,"Adapted from Zublina, 1991 (updated 1997).",Cu_pc,0.0007


In [42]:
crop_nutr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63999 entries, 0 to 63998
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   original_crop                       63999 non-null  object 
 1   item                                62465 non-null  object 
 2   item_code                           63797 non-null  float64
 3   item_group_code                     62465 non-null  float64
 4   item_group                          62465 non-null  object 
 5   factor                              62465 non-null  float64
 6   hs_code                             0 non-null      float64
 7   hs07_code                           62465 non-null  object 
 8   hs12_code                           62465 non-null  object 
 9   cpc_code                            62465 non-null  float64
 10  original_crop_component             63982 non-null  object 
 11  crop_component                      63999

In [43]:
crop_nutr['un_region_name'].unique()

array([nan, 'Americas', 'Asia', 'Europe', 'Africa', 'Oceania'],
      dtype=object)

In [44]:
crop_nutr['un_sub_region_name'].unique()

array([nan, 'Northern America', 'Eastern Asia', 'Eastern Europe',
       'Northern Europe', 'Southern Asia', 'Sub-Saharan Africa',
       'Australia and New Zealand', 'Latin America and the Caribbean',
       'Southern Europe'], dtype=object)

In [45]:
australia_data = crop_nutr[crop_nutr['un_sub_region_name'] == 'Australia and New Zealand']
australia_data.head()


,original_crop,item,item_code,item_group_code,item_group,factor,hs_code,hs07_code,hs12_code,cpc_code,...,un_region_name,un_sub_region_name,un_intermediate_region_name,latitude,longitude,reference_where_data_were_collated,website_of_source_of_collated_data,primary_reference_of_dataset,variable,value
308,Barley,Barley,44.0,1714.0,Crops Primary,1.0,NaN,100300,"100310, 100390",115.0,...,Oceania,Australia and New Zealand,NaN,NaN,NaN,Unkovich et al (2010) A review of biological y...,https://www.sciencedirect.com/science/article/...,Receival standard,Water_pc,11.0
341,Barley,Barley,44.0,1714.0,Crops Primary,1.0,NaN,100300,"100310, 100390",115.0,...,Oceania,Australia and New Zealand,NaN,NaN,NaN,Unkovich et al (2010) A review of biological y...,https://www.sciencedirect.com/science/article/...,Unkovich et al (2006) Australian Crop Yields a...,Mean_Above_ground_biomass_kg_DM_ha,7777.0
426,Barley,Barley,44.0,1714.0,Crops Primary,1.0,NaN,100300,"100310, 100390",115.0,...,Oceania,Australia and New Zealand,NaN,NaN,NaN,Unkovich et al (2010) A review of biological y...,https://www.sciencedirect.com/science/article/...,Unkovich et al (2006) Australian Crop Yields a...,Maximum_Above_ground_biomass_kg_DM_ha,19300.0
454,Barley,Barley,44.0,1714.0,Crops Primary,1.0,NaN,100300,"100310, 100390",115.0,...,Oceania,Australia and New Zealand,NaN,NaN,NaN,Unkovich et al (2010) A review of biological y...,https://www.sciencedirect.com/science/article/...,Unkovich et al (2006) Australian Crop Yields a...,n_Above_ground_biomass_kg_DM_ha,96.0
603,Barley,Barley,44.0,1714.0,Crops Primary,1.0,NaN,100300,"100310, 100390",115.0,...,Oceania,Australia and New Zealand,NaN,NaN,NaN,Unkovich et al (2010) A review of biological y...,https://www.sciencedirect.com/science/article/...,Unkovich et al (2006) Australian Crop Yields a...,Maximum_Grain_yield_kg_DM_ha,8180.0


In [46]:
australia_data['item'].unique()

array(['Barley', 'Rapeseed', 'Chick peas', 'Vegetables, leguminous nes',
       'Cow peas, dry', 'Lentils', 'Lupins', 'Maize', 'Beans, dry',
       'Oats', 'Groundnuts, with shell', 'Rice, paddy', 'Sorghum',
       'Soybeans', 'Sunflower seed', 'Triticale', 'Vetches', 'Wheat'],
      dtype=object)

In [47]:
print(australia_data.columns)

Index(['original_crop', 'item', 'item_code', 'item_group_code', 'item_group',
       'factor', 'hs_code', 'hs07_code', 'hs12_code', 'cpc_code',
       'original_crop_component', 'crop_component', 'original_region',
       'iso3_code', 'country_iso3c', 'un_region_name', 'un_sub_region_name',
       'un_intermediate_region_name', 'latitude', 'longitude',
       'reference_where_data_were_collated',
       'website_of_source_of_collated_data', 'primary_reference_of_dataset',
       'variable', 'value'],
      dtype='object')


In [48]:
australia_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, 308 to 63414
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   original_crop                       543 non-null    object 
 1   item                                543 non-null    object 
 2   item_code                           543 non-null    float64
 3   item_group_code                     543 non-null    float64
 4   item_group                          543 non-null    object 
 5   factor                              543 non-null    float64
 6   hs_code                             0 non-null      float64
 7   hs07_code                           543 non-null    object 
 8   hs12_code                           543 non-null    object 
 9   cpc_code                            543 non-null    float64
 10  original_crop_component             543 non-null    object 
 11  crop_component                      543 non-nu